In [5]:
from keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [6]:
src_path_training = "geodataset/training/"
src_path_test = "geodataset/test/"
classes = ["canada", "japan", "sweden", "united_states", "france"]

train_datagen = ImageDataGenerator(
    rescale  = 1 / 255.0,
    rotation_range = 5,
    zoom_range = 0.01,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    horizontal_flip = True,
    fill_mode = "nearest"
)

test_datagen = ImageDataGenerator(rescale = 1 / 255.0)

In [11]:
batch_size = 20
train_generator = train_datagen.flow_from_directory(
    directory=src_path_training,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_directory(
    directory=src_path_training,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 500 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 100 images belonging to 1 classes.


In [8]:
def prepare_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

In [12]:
model = prepare_model()
model.fit(train_generator, 
                    validation_data=train_generator, 
                    steps_per_epoch=train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=18)

C:\Users\wesle\PycharmProjects\locationclassifier\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 502ms/step - accuracy: 0.1522 - loss: 1.4144 - val_accuracy: 0.2100 - val_loss: 0.5066
Epoch 2/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2080 - val_loss: 0.5066
Epoch 3/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 511ms/step - accuracy: 0.2661 - loss: 0.4989 - val_accuracy: 0.3340 - val_loss: 0.4743
Epoch 4/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3440 - val_loss: 0.4734
Epoch 5/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 491ms/step - accuracy: 0.3261 - loss: 0.4803 - val_accuracy: 0.3400 - val_loss: 0.4797
Epoch 6/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3500 - val_loss: 0.4770
Epoch 7/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 486ms/step - accuracy: 0.3512 - loss: 0.4745 - val_accuracy: 0.3820 - val_loss: 0.4539
Epoch 8/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - accuracy: 0.0000e+00 -

In [13]:
predict = model.predict(test_generator)
y_classes = predict.argmax(axis=-1)
files = test_generator.filenames
for i, p in enumerate(predict):
    index=np.argmax(p)
    classname = classes[index]
    prob = p[index]
    print("for file ", files[i], ' predicted class is ', classname, ' with probability ', prob)

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
for file  predict\canada_0.jpg  predicted class is  canada  with probability  0.3754546
for file  predict\canada_1.jpg  predicted class is  france  with probability  0.5310502
for file  predict\canada_10.jpg  predicted class is  sweden  with probability  0.7703638
for file  predict\canada_11.jpg  predicted class is  japan  with probability  0.27844375
for file  predict\canada_12.jpg  predicted class is  canada  with probability  0.43088487
for file  predict\canada_13.jpg  predicted class is  canada  with probability  0.38742056
for file  predict\canada_14.jpg  predicted class is  japan  with probability  0.5849925
for file  predict\canada_15.jpg  predicted class is  france  with probability  0.49733913
for file  predict\canada_16.jpg  predicted class is  france  with probability  0.76302373
for file  predict\canada_17.jpg  predicted class is  france  with probability  0.6434376
for file  predict\canada_18.jpg  predicted class is  france  with pr